In [66]:
import sys
import pandas as pd
sys.path.insert(1, '/Users/max/projects/machine-learning-labs/tensorflow')
from mytensorflow import *
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfdt
import pandas as pd
import numpy as np
import os
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model

from  matplotlib import pyplot as plt
import random
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from tensorflow import keras
import matplotlib.image as mpimg
%matplotlib inline

In [67]:
IMG_WIDTH=200
IMG_HEIGHT=200
img_folder=r'/Users/max/projects/machine-learning-labs/data/catsdogs'

In [80]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    "../data/catsdogs",
    validation_split=0.2, 
    subset="both", 
    seed=42,
    batch_size=1,
    image_size=(200,200), 
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [88]:
# for item in train_ds:
#     # print(item[0])
train = np.array([item[0] for item in train_ds])
x_train = (train/255)
y_train = np.array([item[0] for item in train_ds])/255
x_train = (train/255)
print(*x_train)


[[[[0.45106617 0.4824387  0.49420342]
   [0.5051593  0.5384927  0.5453554 ]
   [0.4807255  0.52582353 0.5209216 ]
   ...
   [0.30156127 0.2755858  0.23514216]
   [0.29829657 0.23457107 0.20712009]
   [0.3198554  0.27572793 0.24207108]]

  [[0.45781374 0.49702942 0.5048725 ]
   [0.48921567 0.52843136 0.5352941 ]
   [0.49530637 0.53452206 0.5325613 ]
   ...
   [0.2745294  0.23950244 0.2024853 ]
   [0.2801446  0.23207597 0.2046152 ]
   [0.32329166 0.30045587 0.27373773]]

  [[0.4572059  0.50034314 0.5160294 ]
   [0.46607843 0.50333333 0.5120711 ]
   [0.50720584 0.5415196  0.5405392 ]
   ...
   [0.28068626 0.23022059 0.20591912]
   [0.30110294 0.2692157  0.24470589]
   [0.31109068 0.3136397  0.29028186]]

  ...

  [[0.24466915 0.5170098  0.47843137]
   [0.24564955 0.5179902  0.47941175]
   [0.26278192 0.5294118  0.49279416]
   ...
   [0.301875   0.5881495  0.5606985 ]
   [0.30366424 0.58993876 0.5624878 ]
   [0.30553925 0.59181374 0.56436276]]

  [[0.23920587 0.52254903 0.47941175]
   [0.

In [70]:
import os
def create_dataset_PIL(img_folder):
    
    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= np.array(Image.open(image_path))
            image= np.resize(image,(IMG_HEIGHT,IMG_WIDTH,3))
            image = image.astype('float32')
            image /= 255  
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array , class_name  

In [71]:
PIL_img_data, class_name=create_dataset_PIL(img_folder)
len(PIL_img_data)

2000

In [72]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}

In [73]:
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [95]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [103]:
x_train.shape

(2000, 1, 28, 28)

In [97]:
from keras.utils import np_utils

In [101]:
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 200, 200, 3)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

In [102]:
x_train, y_train = preprocess_data(x_train, y_train, 1000)
x_test, y_test = preprocess_data(x_test, y_test, 1000)

ValueError: cannot reshape array of size 1568000 into shape (2000,200,200,3)

In [100]:
net = Network()
net.add(ConvolutionalLayer((200,200,3), (3,3), 1))
net.add(FlattenLayer())
net.add(FCLayer(39204, 1))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
net.compile(BinaryCrossEntropy.fun, BinaryCrossEntropy.derivative)
net.fit(x_train, y_train, epochs=10, learning_rate=0.001)

ValueError: can only specify one unknown dimension

In [91]:
img = tf.keras.preprocessing.image.load_img(
    "../data/catsdogs/cats/0.jpg", target_size=(200, 200)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = net.predict(PIL_img_data[::10])
predictions

[array([0.97968798]),
 array([0.99469695]),
 array([4.29526618e-06]),
 array([0.99469695]),
 array([0.99998472]),
 array([1.]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.95719946]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.99999467]),
 array([0.99469695]),
 array([0.99658559]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.00748332]),
 array([0.99469695]),
 array([0.9901345]),
 array([0.99935446]),
 array([1.]),
 array([0.99986409]),
 array([0.99999969]),
 array([0.08353509]),
 array([0.99999998]),
 array([0.99469695]),
 array([1.]),
 array([0.99998]),
 array([0.99469695]),
 array([0.77167306]),
 array([0.92764328]),
 array([0.99469695]),
 array([0.99119397]),
 array([0.99977648]),
 array([0.17453209]),
 array([0.99469695]),
 array([0.11725765]),
 array([0.99469695]),
 array([0.99469695]),
 array([0.99752402]),
 array([1.]),
 array([0.52725388]),
 array([0.99055995]),
 array([0.99421531])

In [94]:
for y_pred, y_true in zip(predictions, class_name[::10]):
    print(f"pred: {(y_pred)}, true: {y_true}")

pred: [0.97968798], true: dogs
pred: [0.99469695], true: dogs
pred: [4.29526618e-06], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99998472], true: dogs
pred: [1.], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.95719946], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99999467], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99658559], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.99469695], true: dogs
pred: [0.00748332], true: dogs
pred: [0.99469695], true: dogs
pred: [0.9901345], true: dogs
pred: [0.99935446], true: dogs
pred: [1.], true: dogs
pred: [0.99986409], true: dogs
pred: [0.99999969], true: dogs
pred: [0.08353509], true: dogs
pred: [0.99999998], true: dogs
pred: [0.99469695], true: dogs
pred: [1.], true: dogs
pred: [0.99998], true: dogs
pred: [0.99469695], true: dogs
pred: [0.77167306], true: dogs
p